In [1]:
# ---------------------------------------------------------
# 0. IMPORTS
# ---------------------------------------------------------
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# ---------------------------------------------------------
# 1. SETUP & CONFIGURATION
# ---------------------------------------------------------

# Define the model identifiers (NEW MODELS)
MODEL_ID_QWEN = "Qwen/Qwen2.5-3B-Instruct"           # A, F, G
MODEL_ID_GEMMA = "unsloth/Llama-3.2-3B-Instruct"     # B, D
MODEL_ID_LFM   = "unsloth/gemma-3-4b-it"             # C, E

print("Loading Dataset...")
dataset = load_dataset("glue", "sst2")
val_data = dataset["validation"].select(range(50))  # First 50 samples (change if you want)

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(f"CUDA available: {USE_CUDA}")
print(f"Running on: {DEVICE}")

Loading Dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

CUDA available: True
Running on: cuda


In [2]:
# ---------------------------------------------------------
# 2. MODEL LOADING
# ---------------------------------------------------------

def load_model_and_tokenizer(model_id):
    """
    Load model and tokenizer using AutoClasses, ensuring pad token is set.
    Models stay on CPU; we'll move them to GPU only when needed.
    """
    print(f"Loading {model_id}...")
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if USE_CUDA else torch.float32,
        trust_remote_code=True
    )
    model.eval()

    # Ensure pad token is set
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer

print("Loading Models (This may take a moment)...")
model_qwen, tok_qwen   = load_model_and_tokenizer(MODEL_ID_QWEN)
model_gemma, tok_gemma = load_model_and_tokenizer(MODEL_ID_GEMMA)
model_lfm, tok_lfm     = load_model_and_tokenizer(MODEL_ID_LFM)

# Map Logical Agents to Physical Model/Tokenizer pairs
# Architecture:
# Layer 1: A(Qwen), B(Gemma), C(LFM)
# Layer 2: D(Gemma), E(LFM), F(Qwen)
# Layer 3: G(Qwen)
agents = {
    "A": (model_qwen,  tok_qwen),
    "B": (model_gemma, tok_gemma),
    "C": (model_lfm,   tok_lfm),
    "D": (model_gemma, tok_gemma),
    "E": (model_lfm,   tok_lfm),
    "F": (model_qwen,  tok_qwen),
    "G": (model_qwen,  tok_qwen),
}

Loading Models (This may take a moment)...
Loading Qwen/Qwen2.5-3B-Instruct...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

In [3]:
# ---------------------------------------------------------
# 3. HELPER FUNCTIONS
# ---------------------------------------------------------

def parse_label_and_reason(raw_text):
    """
    Parse model output into:
    - label_int: 1 (Positive), 0 (Negative), or -1 (Unknown)
    - label_str: 'Positive', 'Negative', or 'Unknown'
    - reason_str: explanation text (may be whole raw_text as fallback)

    Expected (but not strictly required) format:
        Label: Positive
        Reason: ...
    """
    if not raw_text:
        return -1, "Unknown", ""

    # Split into non-empty lines
    lines = [l.strip() for l in raw_text.splitlines() if l.strip()]
    if not lines:
        return -1, "Unknown", ""

    # First line -> label
    first_line = lines[0].lower()
    if "positive" in first_line and "negative" not in first_line:
        label_int = 1
        label_str = "Positive"
    elif "negative" in first_line and "positive" not in first_line:
        label_int = 0
        label_str = "Negative"
    else:
        label_int = -1
        label_str = "Unknown"

    # Everything after first line -> reason
    if len(lines) > 1:
        reason_lines = []
        for l in lines[1:]:
            # strip "Reason:" prefix if present
            if l.lower().startswith("reason:"):
                l = l.split(":", 1)[1].strip()
            reason_lines.append(l)
        reason_str = " ".join(reason_lines).strip()
    else:
        reason_str = ""

    # Fallback: if reason is empty, keep full raw output as reason
    if not reason_str:
        reason_str = raw_text.strip()

    return label_int, label_str, reason_str


def query_agent(agent_name, messages, max_new_tokens=128):
    """
    Query a specific agent.
    On GPU: temporarily move the model to CUDA for this call, then back to CPU
    to avoid keeping multiple large models in VRAM at once.
    """
    model, tokenizer = agents[agent_name]

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # Move model to the right device for this call
    model.to(device)

    # Build chat prompt as plain string (respecting model's chat template)
    prompt_str = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Make sure inputs are on the same device as the model
    inputs = tokenizer(prompt_str, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,         # greedy
            pad_token_id=tokenizer.pad_token_id
        )

    generated_text = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    ).strip()

    # Move model back to CPU to free VRAM for other models
    if use_cuda:
        model.to("cpu")
        torch.cuda.empty_cache()

    return generated_text

In [4]:
# ---------------------------------------------------------
# 4. MOA ARCHITECTURE IMPLEMENTATION
# ---------------------------------------------------------

results = []

print("\n--- Starting 3-Layer MOA Inference Pipeline ---\n")

for i, item in enumerate(val_data):
    text = item['sentence']
    true_label = item['label']          # 0/1
    true_label_str = "positive" if true_label == 1 else "negative"

    print(f"Sample {i+1}: '{text}' (True: {true_label_str})")

    # ==========================================
    # LAYER 1: Base Agents (A, B, C)
    # Each returns label + reasoning
    # ==========================================

    l1_system = (
        "You are a sentiment classifier for movie reviews.\n"
        "You MUST output EXACTLY TWO lines and nothing else.\n\n"
        "Line 1 format:\n"
        "Label: Positive\n"
        "OR\n"
        "Label: Negative\n\n"
        "Line 2 format:\n"
        "Reason: <one short sentence explaining why>\n\n"
        "Example output:\n"
        "Label: Negative\n"
        "Reason: The review describes the film as boring and disappointing.\n\n"
        "Always include BOTH lines. Never omit the Reason line."
    )
    l1_user = f"Review: {text}\n\nClassify the sentiment."

    l1_messages = [
        {"role": "system", "content": l1_system},
        {"role": "user", "content": l1_user}
    ]

    raw_A = query_agent("A", l1_messages, max_new_tokens=64)
    raw_B = query_agent("B", l1_messages, max_new_tokens=64)
    raw_C = query_agent("C", l1_messages, max_new_tokens=64)

    A_label_int, A_label_str, A_reason = parse_label_and_reason(raw_A)
    B_label_int, B_label_str, B_reason = parse_label_and_reason(raw_B)
    C_label_int, C_label_str, C_reason = parse_label_and_reason(raw_C)

    # ==========================================
    # LAYER 2: Secondary Agents (D, E, F)
    # D uses A, E uses B, F uses C
    # ==========================================

    def build_l2_messages(prev_agent_name, prev_label_str, prev_reason):
        sys_msg = (
            "You are a second-layer sentiment analyst.\n"
            "You see the original review and a previous model's prediction and its reasoning.\n"
            "You may either confirm or correct the previous label.\n"
            "You MUST output EXACTLY TWO lines and nothing else.\n\n"
            "Line 1 format:\n"
            "Label: Positive\n"
            "OR\n"
            "Label: Negative\n\n"
            "Line 2 format:\n"
            "Reason: <one short sentence explaining your decision>\n\n"
            "Example output:\n"
            "Label: Positive\n"
            "Reason: The analysis is correct and the review expresses clear enjoyment.\n\n"
            "Always include BOTH lines."
        )
        user_msg = (
            f"Original review: {text}\n\n"
            f"Previous agent {prev_agent_name} predicted: {prev_label_str}\n"
            f"Reason from {prev_agent_name}: {prev_reason}\n\n"
            "Now, reconsider the sentiment and output your final label and reasoning."
        )
        return [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg}
        ]

    raw_D = query_agent("D", build_l2_messages("A", A_label_str, A_reason), max_new_tokens=64)
    raw_E = query_agent("E", build_l2_messages("B", B_label_str, B_reason), max_new_tokens=64)
    raw_F = query_agent("F", build_l2_messages("C", C_label_str, C_reason), max_new_tokens=64)

    D_label_int, D_label_str, D_reason = parse_label_and_reason(raw_D)
    E_label_int, E_label_str, E_reason = parse_label_and_reason(raw_E)
    F_label_int, F_label_str, F_reason = parse_label_and_reason(raw_F)

    # ==========================================
    # LAYER 3: Aggregator (G)
    # Aggregates D/E/F, sees all labels + reasons + original text
    # ==========================================

    l3_system = (
        "You are an aggregator in a multi-agent sentiment system.\n"
        "You MUST output EXACTLY TWO lines and NOTHING ELSE.\n\n"
        "CRITICAL RULE:\n"
        "You are ONLY allowed to output one of the following labels:\n"
        "  Label: Positive\n"
        "  Label: Negative\n"
        "You are NOT allowed to output any other label.\n\n"
        "FORMAT:\n"
        "Line 1: Label: Positive   OR   Label: Negative\n"
        "Line 2: Reason: <one short sentence explaining your decision>\n\n"
        "Example:\n"
        "Label: Positive\n"
        "Reason: The majority of agents identify a positive sentiment and the review expresses clear enjoyment.\n\n"
        "IMPORTANT: Never output anything other than 'Positive' or 'Negative'."
    )

    l3_user = (
        f"Original review: {text}\n\n"
        f"Agent D -> Label: {D_label_str}, Reason: {D_reason}\n"
        f"Agent E -> Label: {E_label_str}, Reason: {E_reason}\n"
        f"Agent F -> Label: {F_label_str}, Reason: {F_reason}\n\n"
        "Based on these, choose the best final sentiment label and explain briefly."
    )

    l3_messages = [
        {"role": "system", "content": l3_system},
        {"role": "user", "content": l3_user}
    ]

    final_raw = query_agent("G", l3_messages, max_new_tokens=64)
    final_label_int, final_label_str, final_reason = parse_label_and_reason(final_raw)

    is_correct = (final_label_int == true_label)

    # Store Result (with reasoning track)
    results.append({
        "text": text,
        "true_label": true_label_str,
        "L1": {
            "A": {"raw": raw_A, "label": A_label_str, "reason": A_reason},
            "B": {"raw": raw_B, "label": B_label_str, "reason": B_reason},
            "C": {"raw": raw_C, "label": C_label_str, "reason": C_reason},
        },
        "L2": {
            "D": {"raw": raw_D, "label": D_label_str, "reason": D_reason},
            "E": {"raw": raw_E, "label": E_label_str, "reason": E_reason},
            "F": {"raw": raw_F, "label": F_label_str, "reason": F_reason},
        },
        "Final": {
            "raw": final_raw,
            "label": final_label_str,
            "reason": final_reason,
        },
        "Correct": is_correct
    })

    # --------- Detailed trace printing ---------
    print("\n--- Layer 1 (Base Agents) ---")
    print(f"   A -> Label: {A_label_str}, Reason: {A_reason}")
    print(f"   B -> Label: {B_label_str}, Reason: {B_reason}")
    print(f"   C -> Label: {C_label_str}, Reason: {C_reason}")

    print("\n--- Layer 2 (Secondary Agents) ---")
    print(f"   D (from A) -> Label: {D_label_str}, Reason: {D_reason}")
    print(f"   E (from B) -> Label: {E_label_str}, Reason: {E_reason}")
    print(f"   F (from C) -> Label: {F_label_str}, Reason: {F_reason}")

    print("\n--- Layer 3 (Aggregator) ---")
    print(f"   G (final)  -> Label: {final_label_str}, Reason: {final_reason}")
    print(f"\n   ==> FINAL: {final_label_str.upper()} "
          f"[{'CORRECT' if is_correct else 'WRONG'}]  (true: {true_label_str})")
    print("-" * 80)


--- Starting 3-Layer MOA Inference Pipeline ---

Sample 1: 'it 's a charming and often affecting journey . ' (True: positive)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Layer 1 (Base Agents) ---
   A -> Label: Positive, Reason: The review describes the film as charming and affecting, indicating a positive sentiment.
   B -> Label: Positive, Reason: The review uses positive adjectives like "charming" to describe the film.
   C -> Label: Unknown, Reason: 

--- Layer 2 (Secondary Agents) ---
   D (from A) -> Label: Positive, Reason: The review uses words like "charming" and "affecting", which convey a strong positive emotional tone.
   E (from B) -> Label: Unknown, Reason: 
   F (from C) -> Label: Positive, Reason: The review expresses admiration and emotional connection with the journey described.

--- Layer 3 (Aggregator) ---
   G (final)  -> Label: Positive, Reason: The review uses positive descriptors such as "charming" and "affecting," indicating a favorable sentiment.

   ==> FINAL: POSITIVE [CORRECT]  (true: positive)
--------------------------------------------------------------------------------
Sample 2: 'unflinchingly bleak and desperate 

In [5]:
# ---------------------------------------------------------
# 5. SUMMARY
# ---------------------------------------------------------
correct_count = sum(1 for r in results if r['Correct'])
total = len(results)
acc = correct_count / total if total > 0 else 0.0
print(f"\nFinal MOA Accuracy on {total} samples: {correct_count}/{total} ({acc*100:.1f}%)")


Final MOA Accuracy on 50 samples: 47/50 (94.0%)
